<h1>Fermi-Hubbard Hamiltonian (1d & 2d)</h1>

<hr>
DISTRIBUTION STATEMENT A. Approved for public release. Distribution is unlimited.

This material is based upon work supported by the Under Secretary of Defense for
Research and Engineering under Air Force Contract No. FA8702-15-D-0001. Any opinions,
findings, conclusions or recommendations expressed in this material are those of the
author(s) and do not necessarily reflect the views of the Under Secretary of Defense
for Research and Engineering.

© 2023 Massachusetts Institute of Technology.

The software/firmware is provided to you on an As-Is basis

Delivered to the U.S. Government with Unlimited Rights, as defined in DFARS Part
252.227-7013 or 7014 (Feb 2014). Notwithstanding any copyright notice, U.S. Government
rights in this work are defined by DFARS 252.227-7013 or DFARS 252.227-7014 as detailed
above. Use of this work other than as specifically authorized by the U.S. Government
may violate any copyrights that exist in this work.

<hr><hr>

<hr>
<h2>Generating Hamiltonians</h2>

`pyLIQTR` can represent Hamiltonians as a sum of Pauli strings via the `lcu_strings` class.  The routines within this object can handle a variety of tasks, such as normalizing  Hamiltonians for use with QSP, generating matrix representations of the Hamiltonian operator, preparing input strings for circuit generation, consolidating repeated Pauli strings, or writing Hamiltonians to a volume for later use.  As one might surmise from its name, this class requires that the Hamiltonian  can be decomposed into a linear combination of unitary operators (LCU):

\begin{equation}
H = \sum_k \alpha_k U_k,
\end{equation}

where $U_k$ is a unitary operator and $\alpha_k \in \mathbb{R}$ is its associated coefficient.  This holds by construction for our FH Hamiltonian.

To initiate our numerical approach, we must load a few required modules:

In [1]:
import numpy              as  np

import operator_strings   as  os
import hamiltonians       as  hams

We also specify parameters for our Fermi-Hubbard example:

<hr>
<h2>Generating Circuits</h2>

<b>Step 1: Operator Rescaling</b>

QSP-based simulations require a block encoding for the Hamiltonian $H$, which is only well-defined if $H$ satisfies the norm condition $|| H || \leq 1$.  Since we are using an LCU as input, it is convenient to rescale this operator so that $H \mapsto H / \alpha$, where $\alpha = \sum_j | \alpha_j |$ is a 1-norm defined by its coefficients.   To apply this, we will create an `lcu_strings` object for a Fermi-Hubbard instance:

In [2]:
tfactor = 1
N    =    8
N_x  =    N;      N_y  =    N

J    =  1.0;      U    =  2.0
mu   =  0.0;      hz   =  0.0

fh2d_lcu  =  hams.hamiltonian_fh_2d_strings(N_x=N_x,N_y=N_y,J=J,U=U,hz=hz,mu=mu)

alpha = fh2d_lcu.get_alpha()                   # Hamiltonian rescaling factor

t_max = N * tfactor                                  # maximal simulation time

We then perform cleanup steps and conduct rescaling via:

In [3]:
fh2d_lcu.cleanup()

norm_0 = fh2d_lcu.get_coeff_norm()
fh2d_lcu.rescale_coeff_norm()
norm_1 = fh2d_lcu.get_coeff_norm()

print("Norm before rescaling:  ", norm_0)
print("Norm after rescaling:   ", norm_1)

print("Rescaling factor", fh2d_lcu.get_alpha())

Norm before rescaling:   352.0
Norm after rescaling:    1.0000000000000002
Rescaling factor 352.0


The rescaling factor is saved in the `lcu_strings` object (immutably after the first rescaling call), and it is accessible via `get_alpha()`. We must similarly rescale our timesteps so dynamics generated by the propagator $\exp[-\imath H t] = \exp[\imath (H/\alpha) \cdot (\alpha t)]$ remain invariant:

For now, we will skip the operator-level analysis of the QSP sequence and move directly to circuit generation.  This 4x4 instance has already crossed beyond a trivial execution time, and simulations are best conducted using more sophisticated computational techniques (these will be provided later using auxiliary notebooks, scripts, and reference data).

<b>Step 2: Circuit generation.</b>

We approach circuit generation by loading the required components of `pyLIQTR`:

In [4]:

import cirq

from   cirq.contrib.svg        import SVGCircuit

from   pyLIQTR.QSP             import gen_qsp         as   qspFuncs
from   pyLIQTR.QSP             import QSP             as   pQSP
from   pyLIQTR.QSP.gen_qsp     import QSP_Simulator

from   pyLIQTR.QSP.Hamiltonian import Hamiltonian     as pyH
from   pyLIQTR.QSP.qsp_helpers import get_state_vector
from   pyLIQTR.QSP.qsp_helpers import qsp_decompose_once, print_to_openqasm, prettyprint_qsp_to_qasm 

from   pyLIQTR.gate_decomp.cirq_transforms import clifford_plus_t_direct_transform


The LCU that we constructed above must transferred into a dedicated Hamiltonian object for use with `pyLIQTR`'s circuit generation methods.  We do this by calling `pyLIQTR.QSP.Hamiltonian` (which we have aliased as `pyH`) with the rescaled input set: 

In [5]:
lcu_string_set  =  fh2d_lcu.strings()
qsp_H           =  pyH(lcu_string_set)

The next step is to generate QSP phase angles for our time evolution.  A dedicated set of routines in the circuit-generation code handles this task (though they will soon be deprecated in favor of the methods in `Angler`).  Unfortunately these methods can become cumbersome for large problems, at high precision, or for long timesteps.  If we are not interested in simulating the circuit $-$ but only in performing resource analysis $-$ we do not need to compute the actual phases. Instead, we can use analytical bounds to estimate an equivalent number of random phases.  This requires the use of an additional keyword (`mode="random"`) in our input:


In [6]:
required_precision = 1e-2

angles = []

angles.append(qspFuncs.compute_hamiltonian_angles(qsp_H, t_max, required_precision, mode="random")[0])

print ("time","\t","# of required phases")
print ("===============================")
for n in range(len(angles)):
    print(t_max, "\t", len(angles[n]))

time 	 # of required phases
8 	 33


A QSP circuit with $N_\text{phi}$ phase angles contains $N_\text{phi} - 1$ repetitions of the Select oracle, which constitues the majority of our computational overhead.  To first order, we can perform an analysis for a single iterate and scale this the number of repetitions.  This is much more efficient than generating the full circuit.

In [7]:
qsp_generator   = pQSP.QSP(phis=angles[0], hamiltonian=qsp_H, target_size=qsp_H.problem_size)
curr_circ       = qsp_generator.circuit()
curr_circ = curr_circ[0:4]

#SVGCircuit(curr_circ)      # uncomment to view circuit

The nature of this repetition is made apparent by decomposing the circuit further:

In [8]:
decomposed_circuit       = cirq.align_left(qsp_decompose_once(curr_circ))

#SVGCircuit(decomposed_circuit)      # uncomment to view circuit

 As a general rule, we will see $N_\text{phi} - 1$ instances of `SelVBase` and `Prepare` (each) and $N_\text{phi} - 3$ repetitions of `MultiCZ` for a circuit containing $N_\text{phi}$ phases.  The remainder are single-qubit Pauli operations.  
 
 We will demonstrate a subcircuit analysis for  `Prepare`-`SelVBase` segment and a `MultiCZ` segment.  In the first stage, we decompose the circuit down to controlled operations (CNOT, Toffolis, etc.):

In [9]:
circ_selv_prep         =  decomposed_circuit[2:4]
circ_selv_prep_cops    =  cirq.align_left(qsp_decompose_once(circ_selv_prep))

#SVGCircuit(circ_selv_prep_cops)      # uncomment to view circuit

In [10]:
circ_multicz           =  decomposed_circuit[7:8]
circ_multicz_cops      =  cirq.align_left(qsp_decompose_once(circ_multicz))

#SVGCircuit(circ_multicz_cops)      # uncomment to view circuit

In [11]:
circ_selv_prep_cliff_t    = cirq.align_left(clifford_plus_t_direct_transform(circ_selv_prep_cops))
circ_multicz_cliff_t      = cirq.align_left(clifford_plus_t_direct_transform(circ_multicz_cops))

In [12]:
import resource_analysis as ran

svp_nt,svp_ntinv = ran.get_t_counts(circ_selv_prep_cliff_t)
mcz_nt,mcz_ntinv = ran.get_t_counts(circ_multicz_cliff_t)

In [13]:
depth_selv_prep_cops      =  ran.get_gate_depth(circ_selv_prep_cops)
depth_selv_prep_cliff_t   =  ran.get_gate_depth(circ_selv_prep_cliff_t)

depth_multicz_cops        =  ran.get_gate_depth(circ_multicz_cops)
depth_multicz_cliff_t     =  ran.get_gate_depth(circ_multicz_cliff_t)

In [14]:
print()
print("Prepare + Select\n================================")
print(" Clifford+T Depth:\t", depth_selv_prep_cliff_t)
print(" T Count:\t\t", svp_nt)
print(" T^dag Count:\t\t", svp_ntinv)
print(" T Total:\t\t", svp_nt + svp_ntinv)

print("\n")
print("MultiCZ\n================================")
print(" Clifford+T Depth:\t", depth_multicz_cliff_t)
print(" T Count:\t\t", mcz_nt)
print(" T^dag Count:\t\t", mcz_ntinv)
print(" T Total:\t\t", mcz_nt + mcz_ntinv)


Prepare + Select
 Clifford+T Depth:	 46415
 T Count:		 14322
 T^dag Count:		 5658
 T Total:		 19980


MultiCZ
 Clifford+T Depth:	 39
 T Count:		 27
 T^dag Count:		 36
 T Total:		 63


In [15]:
#Write it.
import time
t0 = time.time()
with open('circ_selv_prep_cliff_t_N{}.qasm'.format(N),'w') as f:
    print_to_openqasm(f, circ_selv_prep_cliff_t)
t1 = time.time()
with open('circ_multicz_cliff_t_N{}_CpT.qasm'.format(N),'w') as f:
    print_to_openqasm(f, circ_multicz_cliff_t)
t2 = time.time()
print("OPENQASM time = {} {}".format(t1-t0, t2-t1))

OPENQASM time = 1.0678579807281494 0.0032930374145507812
